In [1]:
import numpy as np
import pandas as pd
import sys
import psutil

from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('iris.csv',header=None)

In [3]:
attributes = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
df.columns = attributes

# Detecting no of classes 

In [6]:
no_of_classes = df['class'].nunique()

In [5]:
df['class'].value_counts()

Iris-versicolor    50
Iris-virginica     50
Iris-setosa        50
Name: class, dtype: int64

In [8]:
names_of_classes = list(df['class'].unique())

# Data Pre Processing

In [7]:
Y = df['class']
dummy_Y = pd.get_dummies(Y,sparse=True)

In [8]:
pre_processed_df = pd.merge(df,dummy_Y,left_index=True,right_index=True)

In [9]:
pre_processed_df.drop(['class'],axis=1,inplace=True)

# Data Splitting

In [10]:
def split_train_test(df,label_cols,test_size=0.2):    
    X = df[df.columns.difference(label_cols)]
    y = df[label_cols]
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=test_size, random_state=42)
    return train_x, test_x, train_y, test_y

In [11]:
x_train, x_test, y_train, y_test = split_train_test(df=pre_processed_df,label_cols=names_of_classes)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [12]:
x_train.shape, x_valid.shape, x_test.shape

((90, 4), (30, 4), (30, 4))

In [13]:
y_train.shape, y_valid.shape, y_test.shape

((90, 3), (30, 3), (30, 3))

# Implementing Keras Model

In [14]:
def get_search_space():    
    space = {'num_layers': hp.choice('num_layers',['one_hidden', 'two_hidden']),
                'units1': hp.choice('units1', [32, 64, 128, 256,512]),
                'units2': hp.choice('units2', [32, 64, 128, 256,512]),
                'dropout1': hp.uniform('dropout1', .25,.75),
                'dropout2': hp.uniform('dropout2',  .25,.75),
                'batch_size' : hp.choice('batch_size', [16,32,64,128]),
                'nb_epochs' :  500,
                'optimizer': hp.choice('optimizer',['rmsprop', 'adam', 'nadam','sgd']),
                'activation': hp.choice('activation',['relu','sigmoid']),
                'early_stop_rounds': hp.choice('early_stop_rounds',[10,20,30,40,50]),
            }
    return space

In [15]:
def create_model(params):    
    x_train_temp = x_train.copy() 
    y_train_temp = y_train.copy()
    model = Sequential()
    model.add(Dense(params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))
    if(params['num_layers'] == 'two_hidden'):
        model.add(Dense(params['units2']))
        model.add(Activation(params['activation']))
        model.add(Dropout(params['dropout2']))
    model.add(Dense(y_train_temp.shape[1]))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=params['optimizer'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=params['early_stop_rounds'])
    terminate_nan = keras.callbacks.TerminateOnNaN()
    history = History()
    model.fit(x_train_temp, y_train_temp,
              batch_size=params['batch_size'],
              epochs=500,
              callbacks=[early_stop, terminate_nan, history],
              verbose=0,
              validation_data=(x_valid,y_valid)) 
    [loss, acc] = model.evaluate(x_valid,y_valid, verbose=0)
    global num
    mem = psutil.virtual_memory()
    if(np.isnan(acc)):
        print("{}) Validation set Accuracy: NaN".format(num),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
        num = num + 1
        return {'loss': np.inf, 'status': STATUS_OK, 'params': params}
    print("{}) Validation set Accuracy: {:7.2f}".format(num,acc*100),"\tAvailable Mem:",(mem.available/1024)/1024,"mb")
    num = num + 1
    return {'loss': -acc, 'status': STATUS_OK, 'params': params}

In [16]:
def train_best_model(best_params):   
    print('Training the best selected model...') 
    x_train_temp = x_train.copy() 
    y_train_temp = y_train.copy()
    model = Sequential()
    model.add(Dense(best_params['units1'], input_shape=(x_train_temp.shape[1],)))
    model.add(Activation(best_params['activation']))
    model.add(Dropout(best_params['dropout1']))
    if(best_params['num_layers'] == 'two_hidden'):
        model.add(Dense(best_params['units2']))
        model.add(Activation(best_params['activation']))
        model.add(Dropout(best_params['dropout2']))
    model.add(Dense(y_train_temp.shape[1]))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=best_params['optimizer'])
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=best_params['early_stop_rounds'])
    history = History()
    model.fit(x_train_temp, y_train_temp,
              batch_size=best_params['batch_size'],
              epochs=500,
              callbacks=[early_stop, history],
              verbose=0,
              validation_data=(x_valid,y_valid)) 

    return model

In [25]:
num= 1
trials=Trials()
space = get_search_space()
print("Selecting the best network architecture specifically for your data...")
best = fmin(create_model, space, algo=tpe.suggest, max_evals=1, trials=trials)
best_trials_temp = trials.best_trial['result'] 
best_model_temp = train_best_model(best_trials_temp['params']) 
# scaled_feature_df = pd.concat([x_train,x_valid,x_test])
# label_df = pd.concat([y_train,y_valid,y_test])
# pred_df = make_predictions(model=best_model_temp,df=scaled_feature_df)
# output_df = pd.merge(input_df,pred_df['predictions'].to_frame(),left_index=True,right_index=True)
# return best_model_temp, output_df, test_arg_processed

Selecting the best network architecture specifically for your data...
1) Validation set Accuracy:   90.00 	Available Mem: 4916.13671875 mb
Training the best selected model...


In [26]:
predicted = best_model_temp.predict(x_test)

In [27]:
predicted1 = np.argmax(predicted, axis=1)

In [28]:
y_pred = pd.get_dummies(predicted1)

In [29]:
best_model_temp.evaluate(x_test,y_test)

30/30 [==============================] - 0s 36us/step


[0.06537923216819763, 1.0]

In [30]:
y_pred.columns = y_test.columns

In [40]:
def reverse_dummies(df,label_col):
    predict_col = pd.DataFrame(df.idxmax(axis=1),columns=[label_col + '_prediction'])
    return predict_col

In [42]:
y_pred_reversed = reverse_dummies(y_pred,'class')